In [ ]:
import pymupdf4llm
import json
import pandas as pd
import openai
import os
from loguru import logger

import psycopg2
from pgvector.psycopg2 import register_vector
import numpy as np
from psycopg2.extras import execute_values
import numpy as np
from dotenv import load_dotenv
load_dotenv()

# Read Resume

In [2]:
md_text = pymupdf4llm.to_markdown("matinsajadi.pdf")
print(md_text)

Processing matinsajadi.pdf...
[                                        ] (0/2===================[====================                    ] (1/2===================[========================================] (2/2]
# MATIN SAJADI

## Machine Learning Engineer

##### 09103323226 09103323226 Yazd, Ardakan

 emtinsajadi79@gmail.com matinsajadi


## HIGHLIGHTS

 I am a c omputer engineering graduate with the ability to solve complex challenges and a high motivation to create innovative solutions in the field of artificial intelligence.

#### Payam Noor University Yazd, Iran

## Computer Vision


### Yazd, Ardakan

 Working on projects focusing on machine learning, deep learning, neural networks, and natural language processing activities. matinsajadii

 matinsajadi Large Language Models Computer Vision

## EDUCATION

### Bachelor of Software Engineering

#### Working on projects focusing on machine learning, deep learning, neural networks, and natural language processing activities.

### Profi

In [3]:
import pathlib

output_file = pathlib.Path("output.md")
output_file.write_bytes(md_text.encode())

4340

# LLM

In [15]:
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="mypass",
    host="localhost",  # or the database server IP address
    port="5432"        # default port for PostgreSQL
)

In [16]:
# Helper function: get embeddings for a text
def get_embeddings(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input = text.replace("\n"," ")
    )
    embedding = response['data'][0]['embedding']
    return embedding

# Connect To Postgres And Retrive Top 3 Similar Jobs

In [17]:
#Helper function: Get top 3 most similar documents from the database
def get_top3_similar_docs(query, conn):
    query_embedding = get_embeddings(query)
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()
    # Get the top 3 most similar documents using the KNN <=> operator
    cur.execute("SELECT * FROM embeddings ORDER BY embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs

In [18]:
query = f"{md_text}"

In [19]:
connection.commit()

In [20]:
get_top3_similar_docs(query=query, conn=connection)

[(35,
  'indeed',
  'https://www.indeed.com/viewjob?jk=8bc2d8079acba213',
  'Senior Software Engineer',
  'Microsoft',
  'Mountain View, CA, US',
  '2024-11-11',
  'yearly',
  117200.0,
  250200.0,
  'USD',
  'true',
  "The Artificial Intelligence (AI) Frameworks team at Microsoft develops AI software that enables running AI models everywhere, from world’s fastest AI supercomputers, to servers, desktops, mobile phones, IoT devices and internet browsers. We collaborate with our hardware teams and partners, both internal and external, and operate at the intersection of AI algorithmic innovation, purpose\\-built AI hardware, systems, and software. We are a team of highly capable and motivated people that pride themselves on a collaborative and inclusive culture.  \n\nWe own inference performance of OpenAI and other state of the art Large Language Models (LLMs) and work directly with OpenAI on the models hosted on the Azure OpenAI service serving some of the largest workloads on the planet